In [14]:
import tensorflow as tf

# List all available physical GPUs
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("GPUs detected:")
    for gpu in gpus:
        print(gpu)
else:
    print("No GPUs detected. TensorFlow will use the CPU.")

# Check if TensorFlow is using the GPU
print("\nRunning a simple computation to verify GPU usage...")

with tf.device('/GPU:0' if gpus else '/CPU:0'):
    # Run a simple matrix multiplication
    a = tf.constant([[1.0, 2.0], [3.0, 4.0]])
    b = tf.constant([[2.0, 0.0], [1.0, 3.0]])
    c = tf.matmul(a, b)

print("Computation result:", c.numpy())


No GPUs detected. TensorFlow will use the CPU.

Running a simple computation to verify GPU usage...
Computation result: [[ 4.  6.]
 [10. 12.]]


In [3]:
import os
import shutil
import glob
import logging
import subprocess
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

from Scripts.ImageAug import process_dataset  # Assuming process_dataset function is defined in ImageAug
from Scripts.ExtractFeatures import extract_features  # Assuming extract_features function is defined in ExtractFeatures
from Scripts.Models import evaluate_models  # Assuming evaluate_models function is defined in Models

2024-11-11 06:09:23.536202: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731305363.554396  209037 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731305363.559843  209037 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-11 06:09:23.578159: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [5]:
def list_directory_contents(directory):
    """List the contents of a directory."""
    logging.info(f"Contents of {directory}:")
    contents = os.listdir(directory)
    for item in contents:
        logging.info(item)

In [6]:
def create_and_process_dataset(dataset_type, num_augmentations=1):
    """Create and process the dataset."""
    os.system(f"./Scripts/create_dataset.sh {dataset_type}")
    logging.info("Dataset creation script executed for %s", dataset_type)
    
    dataset_dir = f"{dataset_type.capitalize()}_Dataset/"
    
    # Check if the dataset directory was created
    if not os.path.exists(dataset_dir):
        logging.error("Dataset directory not found: %s", dataset_dir)
        raise FileNotFoundError(f"Dataset directory '{dataset_dir}' does not exist.")
    
    list_directory_contents(dataset_dir)
    
    process_dataset(dataset_dir, num_augmentations=num_augmentations)
    logging.info("Processing completed for %s", dataset_dir)
    
    return 'Augmented_DataSet/'

In [7]:
def extract_features_for_models(data_dir):
    """Extract features using various models."""
    models = ['ResNet50', 'InceptionV3', 'MobileNetV2', 'DenseNet121', 'EfficientNetB0']
    for model in models:
        extract_features(model, data_dir)
        logging.info("Extracted features using %s", model)

In [8]:
def evaluate_models_and_save_results(model_params, output_filename, class_names, dataset_type):
    """Evaluate models, save results, and save models to directory."""
    results = evaluate_models(model_params=model_params, class_names=class_names)
    
    # Ensure Reports directory exists before renaming
    if not os.path.exists("Reports"):
        os.makedirs("Reports")
    
    # Save the results to the specified output filename
    os.rename("Reports/model_results.csv", output_filename)
    logging.info("Model results saved to %s", output_filename)

    # Move saved models to specific directory for this dataset type
    move_models(dataset_type)

In [9]:
def move_cm(dir_name):
    """Move confusion matrices to a directory named after the dataset type."""
    os.makedirs(f"Conf_Matrix/{dir_name}", exist_ok=True)
    for file_path in glob.glob("Conf_Matrix/*.png"):
        shutil.move(file_path, f"Conf_Matrix/{dir_name}/")
        print(f'Moved: {file_path} to Conf_Matrix/{dir_name}/')

In [10]:
def move_models(dir_name):
    """Move saved models to a directory named after the dataset type."""
    os.makedirs(f"Models/{dir_name}", exist_ok=True)
    for file_path in glob.glob("Models/*.pkl"):  # Assuming models are saved as .pkl files
        shutil.move(file_path, f"Models/{dir_name}/")
        print(f'Moved: {file_path} to Models/{dir_name}/')

In [11]:
def main():
    # List contents of Original_DataSet
    list_directory_contents("Original_DataSet/")

    # Define model parameters
    model_params = {
        SVC: [{'kernel': 'rbf'}, {'kernel': 'linear'}],
        DecisionTreeClassifier: [{'max_depth': None}, {'max_depth': 5}],
        RandomForestClassifier: [{'n_estimators': 100, 'max_depth': None, 'random_state': 42}],
        KNeighborsClassifier: [{'n_neighbors': 5}]
    }

    # Dataset configurations: dataset type, result filename, and class names for each type
    datasets = [
        {'type': 'binary', 'filename': 'Reports/model_results_binary.csv', 'class_names': ['Healthy', 'Reject']},
        {'type': 'multiclass1', 'filename': 'Reports/model_results_three_class.csv', 'class_names': ['Ripe', 'Unripe', 'Reject']},
        {'type': 'multiclass2', 'filename': 'Reports/model_results_four_class.csv', 'class_names': ['Ripe', 'Unripe', 'Old', 'Damaged']}
    ]

    # Create, process datasets, extract features, evaluate models, and organize results
    for dataset in datasets:
        data_dir = create_and_process_dataset(dataset['type'])
        extract_features_for_models(data_dir)
        evaluate_models_and_save_results(
            model_params=model_params,
            output_filename=dataset['filename'],
            class_names=dataset['class_names'],
            dataset_type=dataset['type']
        )
        move_cm(dataset['type'])

    logging.info("#################### Completed ##########################")
    # Command to remove the specified directories
    command = "rm -rf Augmented_DataSet Binary_Dataset Features Multiclass1_Dataset Multiclass2_Dataset"

    # Execute the command in a shell
    try:
        subprocess.run(command, shell=True, check=True)
        print("Directories removed successfully.")
    except subprocess.CalledProcessError as e:
        print(f"Error occurred: {e}")

In [ ]:
if __name__ == "__main__":
    main()

2024-11-11 06:09:30,876 - INFO - Contents of Original_DataSet/:
2024-11-11 06:09:30,879 - INFO - Unripe
2024-11-11 06:09:30,880 - INFO - Old
2024-11-11 06:09:30,881 - INFO - Ripe
2024-11-11 06:09:30,882 - INFO - Damaged
2024-11-11 06:09:31,062 - INFO - Dataset creation script executed for binary
2024-11-11 06:09:31,064 - INFO - Contents of Binary_Dataset/:
2024-11-11 06:09:31,065 - INFO - Healthy
2024-11-11 06:09:31,066 - INFO - Reject


Binary Dataset creation Completed
Saved Augmented_DataSet/Healthy/augmented_1_UNRIPE-025.png
Saved Augmented_DataSet/Healthy/augmented_1_r (609).png
Saved Augmented_DataSet/Healthy/augmented_1_UNRIPE-472.png
Saved Augmented_DataSet/Healthy/augmented_1_r (666).png
Saved Augmented_DataSet/Healthy/augmented_1_UNRIPE-196.png
Saved Augmented_DataSet/Healthy/augmented_1_r (740).png
Saved Augmented_DataSet/Healthy/augmented_1_r (686).png
Saved Augmented_DataSet/Healthy/augmented_1_UNRIPE-192.png
Saved Augmented_DataSet/Healthy/augmented_1_r (600).png
Saved Augmented_DataSet/Healthy/augmented_1_UNRIPE-141.png
Saved Augmented_DataSet/Healthy/augmented_1_UNRIPE-463.png
Saved Augmented_DataSet/Healthy/augmented_1_r (498).png
Saved Augmented_DataSet/Healthy/augmented_1_r (504).png
Saved Augmented_DataSet/Healthy/augmented_1_r (991).png
Saved Augmented_DataSet/Healthy/augmented_1_UNRIPE-067.png
Saved Augmented_DataSet/Healthy/augmented_1_r (497).png
Saved Augmented_DataSet/Healthy/augmented_1_r (69

2024-11-11 06:10:20,976 - INFO - Processing completed for Binary_Dataset/


Saved Augmented_DataSet/Reject/augmented_1_o (239).png
Saved Augmented_DataSet/Reject/augmented_1_D752.png
Saved Augmented_DataSet/Reject/augmented_1_D112.png
Saved Augmented_DataSet/Reject/augmented_1_D191.png


W0000 00:00:1731305421.164541  209037 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


...

Features and labels extracted using MobileNetV2 and saved successfully in 'Features' directory.
29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
Found 3466 images belonging to 2 classes.
109/109 ━━━━━━━━━━━━━━━━━━━━ 29s 244ms/step


2024-11-11 06:12:05,074 - INFO - Extracted features using DenseNet121


Features and labels extracted using DenseNet121 and saved successfully in 'Features' directory.
16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
Found 3466 images belonging to 2 classes.
109/109 ━━━━━━━━━━━━━━━━━━━━ 17s 142ms/step


2024-11-11 06:12:26,867 - INFO - Extracted features using EfficientNetB0


Features and labels extracted using EfficientNetB0 and saved successfully in 'Features' directory.

 Loaded ResNet50 Features ! 

